# Bus AVL Project

Ehab Ebeid

## Setup

In [1]:
from __future__ import division, print_function

import numpy as np


## Getting the Data

The data is located in a local MS SQL Server database.

In [2]:
import pyodbc 
import pandas as pd

# limit imported data using timestamps of trip start datetime, in the format "yyyy-mm-dd hh:mi:ss", local time.
FROM_DT = '2018-07-11 06:00:00'
TO_DT = '2018-07-11 09:30:00'

# connect to database
cnxn = pyodbc.connect(Trusted_Connection='yes', 
                      driver = '{SQL Server}',
                      server = 'EHAB\SQLEXPRESS', 
                      database = 'raw_nav')

# join route_run and gps_reading tables
query = """select 
            r.id as route_run_id,
            r.route_pattern,
            r.bus_id,
            r.svc_date,
            r.trip_start_date_time,
            g.id as gps_reading_id,
            g.lat_fixed as lat,
            g.lon_fixed as lon,
            g.heading_deg,
            g.door_state,
            g.vehicle_state,
            g.odo_feet,
            g.time_seconds,
            g.sat_count,
            g.stop_window_data

        from raw_nav.dbo.route_run as r
        join raw_nav.dbo.gps_reading as g
        on r.id = g.route_run_id
        
        where trip_start_date_time between '{start}' and '{end}'
        ;
        """.format(start=FROM_DT, end=TO_DT)

# grab as a pandas dataframe
df = pd.read_sql_query(query, cnxn)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47827 entries, 0 to 47826
Data columns (total 15 columns):
route_run_id            47827 non-null int64
route_pattern           47827 non-null object
bus_id                  47827 non-null object
svc_date                47827 non-null object
trip_start_date_time    47827 non-null datetime64[ns]
gps_reading_id          47827 non-null int64
lat                     47827 non-null float64
lon                     47827 non-null float64
heading_deg             47827 non-null int64
door_state              47827 non-null object
vehicle_state           47827 non-null object
odo_feet                47827 non-null int64
time_seconds            47827 non-null int64
sat_count               47827 non-null int64
stop_window_data        1156 non-null object
dtypes: datetime64[ns](1), float64(2), int64(6), object(6)
memory usage: 5.5+ MB


## Cleaning the Data

### Labelling Bad Runs